## Code to label Facebook news sources data for Study 2
Go to Runtime > Change runtime type  and set it to GPU, A100 or V100 and High-RAM for fastest perfromance (consider paying a one-time fee of GBP 10 for 100 computing units or subscribing to ColabPro for a bit for a much faster performance)  

In [ ]:
!pip install transformers sentencepiece

In [3]:
import pandas as pd
import torch
import transformers
from transformers import pipeline
import sklearn
import numpy as np
from sklearn import metrics
from sklearn.metrics import f1_score
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
data = pd.read_csv("/content/sampled_utterances (1).csv")

In [63]:
import re
data["text"] = 'The quote: "' + data.convo.apply(lambda x: re.sub(r"http\S+", "<url>", str(x)), 1) + '" - end of the quote.'

In [9]:

hypothesis_label_dic = {
    "Other": "The quote does not exhibit psr",
    "PSR": "The quote is expressing PSR",
}
hypothesis_lst = list(hypothesis_label_dic.values())

In [56]:
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# documentation: https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.ZeroShotClassificationPipeline
pipe_classifier = pipeline(
    "zero-shot-classification",
    model="mellonn/Parasocial-Language-Classifier",
    framework="pt",
    device=device,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [57]:
print(device)

cuda:0


In [71]:
# create a dummy data frame for illustration
text_lst = data['text'].tolist()

# use the pipeline with your chosen model for inference (prediction)
pipe_output = pipe_classifier(
    text_lst,  # input any list of texts here
    candidate_labels=hypothesis_lst,
    hypothesis_template="{}",
    multi_label=False,  # here you can decide if, for your task, only one hypothesis can be true, or multiple can be true
    batch_size=32  # reduce this number to 8 or 16 if you get an out-of-memory error
)

In [75]:
# extract the predictions from pipe_outut
hypothesis_pred_true_probability = []
hypothesis_pred_true = []
for dic in pipe_output:
    hypothesis_pred_true_probability.append(dic["scores"][0])
    hypothesis_pred_true.append(dic["labels"][0])

# map the long hypotheses to their corresponding short label names
hypothesis_label_dic_inverted = {value: key for key, value in hypothesis_label_dic.items()}
label_pred = [hypothesis_label_dic_inverted[hypo] for hypo in hypothesis_pred_true]

# add inference data to your original dataframe
data["engagement_pred"] = label_pred
data["engagement_pred_proba"] = hypothesis_pred_true_probability

In [76]:
data.to_csv("/content/drive/MyDrive/fb_media_sol.csv")

### Hostility

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/fb_media_sol.csv")

In [ ]:
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda:0'

In [ ]:
hypothesis_label_dic = {
    "Other": "The quote is not about hostility towards Russia or Russians",
    "Hostility towards Russia or Russians": "The quote is expressing hostility, derogation or disliking of Russia or Russians, for example it criticises Russia, mentions Russians as incompetent, immoral people or the Russian Federation as a bad, weak or failing nation.",
}
hypothesis_lst = list(hypothesis_label_dic.values())

In [ ]:
pipe_classifier = pipeline(
    "zero-shot-classification",
    model='STUDY 2 HOSTILITY MODEL',
    framework="pt",
    device=device,
)

In [ ]:

text_lst = data["text_prepared"].tolist()

# use the pipeline with your chosen model for inference (prediction)
pipe_output = pipe_classifier(
    text_lst,  # input any list of texts here
    candidate_labels=hypothesis_lst,
    hypothesis_template="{}",
    multi_label=False,  # here you can decide if, for your task, only one hypothesis can be true, or multiple can be true
    batch_size=32  # reduce this number to 8 or 16 if you get an out-of-memory error
)

In [ ]:
# extract the predictions from pipe_outut
hypothesis_pred_true_probability = []
hypothesis_pred_true = []
for dic in pipe_output:
    hypothesis_pred_true_probability.append(dic["scores"][0])
    hypothesis_pred_true.append(dic["labels"][0])

# map the long hypotheses to their corresponding short label names
hypothesis_label_dic_inverted = {value: key for key, value in hypothesis_label_dic.items()}
label_pred = [hypothesis_label_dic_inverted[hypo] for hypo in hypothesis_pred_true]

# add inference data to your original dataframe
data["hostility_pred"] = label_pred
data["hostility_pred_proba"] = hypothesis_pred_true_probability

In [ ]:
data.to_csv("/content/drive/MyDrive/fb_media_sol_host.csv")